In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
figdir = "../images/"

In [ ]:
# import bayfox
# import erebusfall
# from lgmproxies.datasets.tierney import TIERNEY_REPOS
# TIERNEY_REPOS
import pandas as pd
from lgmproxies.datasets.tierney import get_repo_path
proxydata = get_repo_path("jesstierney/lgmDA")/"proxyData"
! ls {proxydata}
df = pd.read_csv(proxydata/"Tierney2020_ProxyDataPaired.csv")
df
# get_repo_path

In [ ]:
lh = pd.read_csv(proxydata/"Tierney2020_LHProxyData.csv")
lgm = pd.read_csv(proxydata/"Tierney2020_LGMProxyData.csv")

color_map = {
    "delo": "tab:blue",
    "mg": "tab:orange",
    "tex": "tab:green",
    "uk": "tab:red"
}

marker_map = {
    "bulloides": "o",
    "incompta": "s",
    "pachy": "^",
    "ruber": "D",
    "sacculifer": "v",
    "nan": "X"
}

f, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)


for ax, df in zip(axes.flat, [lh, lgm]):
    for proxytype in sorted(df["ProxyType"].unique()):
        df_proxy = df[df["ProxyType"] == proxytype]
        for species in sorted(df_proxy["Species"].unique(), key=lambda x: str(x)):
            if pd.isna(species):
                species_key = "nan"
                df_species = df_proxy[df_proxy["Species"].isna()]
                label = f"{proxytype}"
            else:
                species_key = species
                df_species = df_proxy[df_proxy["Species"] == species]
                label = f"{proxytype} ({species})"
            if len(df_species) > 0:
                ax.scatter(
                    df_species["ProxyValue"], df_species["SSTMedian"],
                    label=label,
                    color=color_map.get(proxytype, "gray"),
                    marker=marker_map.get(species_key, "x"),
                    alpha=0.6,
                    edgecolor="black",
                    linewidth=0.5
                )

    ax.set_xlabel("Proxy Value")
    if ax == axes[0]:
        ax.set_ylabel("SST Median (°C)")
    ax.set_title("LGM Proxies" if ax == axes[1] else "LH Proxies")
    ax.grid(True)
    # only show legend for the last subplot
    if ax == axes[-1]:
        ax.legend(loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

plt.tight_layout()
plt.savefig(figdir+"tierney_calibration.png", dpi=300, bbox_inches='tight')